In [1]:
import pandas as pd
import random
from base64 import b64decode
from json import loads
import numpy as np
import matplotlib.pyplot as plt
# set matplotlib to display all plots inline with the notebook
%matplotlib inline
import math
import os
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

In [2]:
city_years = {
 'Francisco_Bay':[2016,2017,2018,2019,2020,2021,2022,2023],
}

In [3]:
path = '/Users/zahrafarook/Desktop/DataAnalysis/data'
dir_list = os.listdir(path)
print(dir_list)

['cnc', 'Virginia', '.DS_Store', 'San_Franciso', 'France', 'San_Francisco_Bay_2020.csv', 'San_Francisco_Bay_2021.csv', 'census_ethnicities', 'San_Francisco_Bay_2023.csv', 'San_Francisco_Bay_2022.csv', 'Sri_Lanka', 'Los_Angeles_2017.csv', 'San_Francisco_Bay_2019.csv', 'San_Francisco_Bay_2018.csv', 'non_cnc', 'Bolivia', 'San_Francisco_Bay_2016.csv', 'northcarolina', 'UK', 'San_Francisco_Bay_2017.csv', 'Los_Angeles', 'India', 'Texas', 'old data']


In [4]:
dfs = defaultdict(dict)
dfall = []


for city, years in city_years.items():
    for year in years:
        print("Loading: ", city, year)

        df = pd.read_csv("/Users/zahrafarook/Desktop/DataAnalysis/data/San_{}_{}.csv".format(city, year))
       
        #creating a column to add weekend or weekday
        #df['time_observed_at'] = pd.to_datetime(df['time_observed_at'], utc=True)
        #df['day_of_week'] = df['time_observed_at'].dt.dayofweek
        #df['day_type'] = df['time_observed_at'].apply(lambda x: 'Weekend' if x.weekday() in [5, 6] else 'Weekday')
        dfs[city][year] = df
        dfall.append(df)

dfall = pd.concat(dfall)
print(dfall.columns)
dfs['Francisco_Bay'][2016].head()

Loading:  Francisco_Bay 2016
Loading:  Francisco_Bay 2017
Loading:  Francisco_Bay 2018
Loading:  Francisco_Bay 2019
Loading:  Francisco_Bay 2020
Loading:  Francisco_Bay 2021
Loading:  Francisco_Bay 2022
Loading:  Francisco_Bay 2023
Index(['id', 'observed_on_string', 'observed_on', 'time_observed_at',
       'created_time_zone', 'created_at', 'updated_at', 'description',
       'user_id', 'quality_grade', 'reviewed_by', 'faves_count',
       'num_identification_agreements', 'num_identification_disagreements',
       'identifications_most_agree', 'identifications_most_disagree',
       'captive', 'place_guess', 'place_ids', 'longitude', 'latitude',
       'positional_accuracy', 'geoprivacy', 'taxon_geoprivacy', 'obscured',
       'species_guess', 'taxon_id', 'taxon_name', 'preferred_common_name',
       'iconic_taxon_name', 'taxon_rank', 'taxon_parent_id', 'taxon_native',
       'taxon_endemic', 'taxon_threatened', 'taxon_search_rank',
       'taxon_observations', 'identifications'],
   

,id,observed_on_string,observed_on,time_observed_at,created_time_zone,created_at,updated_at,description,user_id,quality_grade,...,preferred_common_name,iconic_taxon_name,taxon_rank,taxon_parent_id,taxon_native,taxon_endemic,taxon_threatened,taxon_search_rank,taxon_observations,identifications
0,20069,1:15 pm.,2016-07-14,2016-07-14T13:15:00-07:00,America/Los_Angeles,2011-06-03T14:51:45-07:00,2020-02-09T08:18:18-08:00,Seen in grass. Individual had >5 ticks from e...,1704,casual,...,Western Alligator Lizards,Reptilia,genus,797512.0,False,False,False,36300.0,36300.0,"[{'user_id': 1704, 'category': 'leading', 'dis..."
1,20070,1:00 pm.,2016-03-25,2016-03-25T13:00:00-07:00,America/Los_Angeles,2011-06-03T14:53:13-07:00,2020-02-09T08:18:11-08:00,Female mallard duck with multiple ducklings.,1704,casual,...,Mallard,Aves,species,6922.0,True,False,False,462859.0,462859.0,"[{'user_id': 642, 'category': 'leading', 'disa..."
2,68373,6:30,2016-02-12,2016-02-12T06:30:00-08:00,America/Los_Angeles,2012-04-20T20:36:48-07:00,2020-08-01T14:11:59-07:00,This flower has blue/purple petals that are di...,5844,casual,...,periwinkles,Plantae,genus,632394.0,False,False,False,98028.0,98028.0,"[{'user_id': 5844, 'category': 'leading', 'dis..."
3,158736,2:19,2016-10-14,2016-10-14T14:19:00-07:00,America/Los_Angeles,2012-12-06T20:23:52-08:00,2016-10-14T01:56:44-07:00,We saw our first devil's coach-horse beetle as...,11548,casual,...,Devil's Coach Horse Beetle,Insecta,species,1270925.0,False,False,False,10365.0,10365.0,"[{'user_id': 11548, 'category': 'leading', 'di..."
4,538018,2016-04-10 2:20:00 PM PDT,2016-04-10,2016-04-10T14:20:00-07:00,America/Los_Angeles,2014-02-20T15:40:40-08:00,2016-04-10T21:06:27-07:00,NaN,16603,research,...,Western Fence Lizard,Reptilia,species,36141.0,True,False,False,119535.0,119535.0,"[{'user_id': 16603, 'category': 'improving', '..."


In [5]:
def get_obs_ident_per_users(dfcity,year):
    """ Calculate the number of observations and identifications per user 
    
    Input:
    ------
     - dfcity: pd.DataFrame
         The observations data for one city.
     - year: int
         The year corresponding to the observations data.
    
    Returns:
    -------
     - dfobsident: pd.DataFrame
         Has the columns: user_id | year | n_obs | n_idents
    """
    
    n_obs = defaultdict(int)
    n_idents = defaultdict(int)
    
    for _, row in dfcity.iterrows():
        # Total count of observation made by each users
        user_obs = row['user_id']
        n_obs[user_obs] += 1 # Increment observation count for the user
        
        # Total identification user made
        identifications = row['identifications']
        if not pd.isnull(identifications):  # Check if 'identifications' is not NaN
            identifications = eval(identifications)
            for ident in identifications:
                user_ident = ident['user_id']
                if user_ident != user_obs:
                    n_idents[user_ident] += 1 # Increment identification count for the user
    
    # Aggregating results into a list of dictionaries
    obs_ident_list = []
    for user_id, obs_count in n_obs.items():
        obs_ident_list.append({
            'user_id': user_id,
            'n_obs': obs_count,
            'n_idents': n_idents[user_id],
            #'day_type': row['day_type'],  # Include the 'day_type' column in the result
            'year': year,
            'time_observed_at': row['time_observed_at']
            
            
        })
    
    # Create DataFrame from the list of dictionaries
    dfobsident = pd.DataFrame(obs_ident_list)
    
    return dfobsident

In [6]:
# Apply the function to each city's data and concatenate the results
dfobsident_list = []
for city, years in city_years.items():
    for year in years:
        dfobsident = get_obs_ident_per_users(dfs[city][year], year)
        dfobsident_list.append(dfobsident)

# Concatenate all observation-identification dataframes into a single dataframe
dfobsident = pd.concat(dfobsident_list, ignore_index=True)

In [ ]:
dfobsident

In [7]:
# Initialize empty dictionary to store results for each year
sf_dataframes = {}

# Iterate over the years
for year in range(2016, 2024):
    # Extract DataFrame for the current year and city 'SF'
    dfcity = dfs['Francisco_Bay'][year]
    # Process data for the current year
    sf_dataframes[year] = get_obs_ident_per_users(dfcity, year)

#access dataframe for each year

In [8]:
sf_2016 = sf_dataframes[2016]
sf_2017 = sf_dataframes[2017]
sf_2018 = sf_dataframes[2018]
sf_2019 = sf_dataframes[2019]
sf_2020 = sf_dataframes[2020]
sf_2021 = sf_dataframes[2021]
sf_2022 = sf_dataframes[2022]
sf_2023 = sf_dataframes[2023]

In [9]:
def transform_data(df):
    # Step before log transformation to avoid the zeros
    df['obs_count_1'] = df['n_obs'] + 1
    df['ident_count_1'] = df['n_idents'] + 1

    # Log transform
    df['obs_count_log'] = np.log(df['obs_count_1'])
    df['ident_count_log'] = np.log(df['ident_count_1'])

    # Select variables for plotting
    df_select = df[['user_id', 'obs_count_log', 'ident_count_log']]

    # Create a copy of the dataframe
    df_std = df_select.copy()

    # Don't include the user_id column in the transformation
    col_names = ['obs_count_log', 'ident_count_log']
    features = df_std[col_names]
    scaler = StandardScaler().fit(features.values)
    features = scaler.transform(features.values)

    # Assign the result to those two columns
    df_std[col_names] = features
    df_std.rename(columns={'obs_count_log': 'obs_count_scaled', 'ident_count_log': 'ident_count_scaled'}, inplace=True)
    df_std.sort_values('user_id', inplace=True)
    
    return df_std

In [10]:
df_transformed = transform_data(dfobsident)

In [11]:
# pick a ratio for splitting the digits list into a training and a validation set.
#digits = list(digits)
#extract features/ column for clustering
data = list(zip(df_transformed['obs_count_scaled'], df_transformed['ident_count_scaled']))

ratio = int(len(data)*0.25)
validation = data[:ratio] #Validation set
training = data[ratio:] #training set


In [12]:
def init_centroids(data, k):
    """
    Randomly pick some k centers from the data as starting values for centroids.
    """
    return random.sample(data, k)

# Define a function to sum a cluster element-wise
def sum_cluster(cluster):
    """
    Element-wise sums a list of arrays.
    """
    if len(cluster) == 0:
        return np.zeros_like(cluster[0])  # Return a zero vector if the cluster is empty
    sum_ = np.array(cluster[0]).copy()
    for vector in cluster[1:]:
        sum_ += vector
    return sum_

# Define a function to compute the mean of a cluster
def mean_cluster(cluster):
    """
    Computes the mean (i.e., the centroid at the middle) of a list of vectors (a cluster).
    Takes the sum and then divides by the size of the cluster.
    """
    if len(cluster) == 0:
        return np.zeros(len(cluster[0]))  # Return a zero vector if the cluster is empty
    sum_of_points = sum_cluster(cluster)
    mean_of_points = sum_of_points * (1.0 / len(cluster))
    return mean_of_points

In [13]:
# Define a function to form clusters
def form_clusters(data, centroids):
    """
    Given some data and centroids for the data, allocate each datapoint
    to its closest centroid. This forms clusters.
    """
    centroids_indices = range(len(centroids))
    clusters = {c: [] for c in centroids_indices}
    
    for point in data:
        smallest_distance = float("inf")
        for cj_index in centroids_indices:
            cj = centroids[cj_index]
            distance = np.linalg.norm(np.array(point) - cj)
            if distance < smallest_distance:
                closest_centroid_index = cj_index
                smallest_distance = distance
        clusters[closest_centroid_index].append(point)
    
    return list(clusters.values())


def move_centroids(clusters, data):
    """
    Returns a list of centroids corresponding to the clusters.
    If a cluster is empty, reinitialize its centroid to a random data point.
    """
    new_centroids = []
    for cluster in clusters:
        if len(cluster) > 0:
            new_centroids.append(mean_cluster(cluster))
        else:
            print("Warning: Empty cluster detected. Reinitializing to a random data point.")
            new_centroids.append(random.choice(data))
    return new_centroids

# Define a function to repeat until convergence
def repeat_until_convergence(data, clusters, centroids):
    """
    Form clusters around centroids, then keep moving the centroids
    until the moves are no longer significant, i.e., we've found
    the best-fitting centroids for the data.
    """
    previous_max_difference = float("inf")
    while True:
        old_centroids = centroids
        centroids = move_centroids(clusters, data)
        clusters = form_clusters(data, centroids)
        
        differences = [np.linalg.norm(np.array(a) - np.array(b)) for a, b in zip(old_centroids, centroids)]
        max_difference = max(differences)
        mean_difference = np.mean([previous_max_difference, max_difference])
        
        if np.isnan(mean_difference) or mean_difference == 0:
            difference_change = 0
        else:
            difference_change = abs((max_difference - previous_max_difference) / mean_difference) * 100
        
        previous_max_difference = max_difference
        
        if difference_change == 0:
            break
    
    return clusters, centroids


In [14]:
# Define a function to run k-means clustering
def cluster(data, k):
    """
    Runs k-means clustering on the data.
    """
    centroids = init_centroids(data, k)
    clusters = form_clusters(data, centroids)
    final_clusters, final_centroids = repeat_until_convergence(data, clusters, centroids)
    return final_clusters, final_centroids


In [15]:
# Define a function to assign labels to centroids
def assign_labels_to_centroids(clusters, centroids):
    """
    Assigns a label to each cluster. In this context, the label can be the centroid itself.
    NOTE: this function depends on clusters and centroids being in the same order.
    """
    labelled_centroids = [(centroids[i], clusters[i]) for i in range(len(clusters))]
    return labelled_centroids

In [16]:
def classify_user(user, labelled_centroids):
    """
    Classifies a user based on the nearest centroid.
    """
    smallest_distance = float('inf')
    closest_label = None
    
    for label, centroid in labelled_centroids:
        distance = np.linalg.norm(np.array(user) - np.array(centroid))
        if distance < smallest_distance:
            smallest_distance = distance
            closest_label = label
    
    return closest_label


def get_error_rate(users, labelled_centroids):
    classified_incorrect = 0
    
    for user in users:
        classified_label = classify_user(user, labelled_centroids)
        if not any(np.array_equal(classified_label, c[0]) for c in labelled_centroids):
            classified_incorrect += 1
    error_rate = classified_incorrect / float(len(users))
    return error_rate



In [ ]:
## recheck this

def get_error_rate(users, labelled_centroids):
    classified_incorrect = 0
    
    # Convert centroids to tuples 
    labelled_centroids_tuples = [(label, tuple(centroid)) for label, centroid in labelled_centroids]
    
    for user in users:
        classified_label = classify_user(user, labelled_centroids)
        
        # Debug print to check classified labels and centroids
        print(f"User: {user}, Classified Label: {classified_label}")
        
        if not any(classified_label == c[0] for c in labelled_centroids_tuples):
            classified_incorrect += 1
    
    error_rate = classified_incorrect / float(len(users))
    
    # Debug print to check the error rate
    print(f"Classified Incorrect: {classified_incorrect}, Total Users: {len(users)}, Error Rate: {error_rate}")
    
    return error_rate


In [ ]:
# Initialize error rates dictionary
error_rates = {x: None for x in list(range(5, 25)) + [100]}

# Calculate error rates for each value of k
for k in range(5, 25):
    trained_clusters, trained_centroids = cluster(training, k)
    labelled_centroids = assign_labels_to_centroids(trained_clusters, trained_centroids)
    error_rate = get_error_rate(validation, labelled_centroids)
    error_rates[k] = error_rate

# Also calculate for k=100
trained_clusters, trained_centroids = cluster(training, 100)
labelled_centroids = assign_labels_to_centroids(trained_clusters, trained_centroids)
error_rate = get_error_rate(validation, labelled_centroids)
error_rates[100] = error_rate

# Show the error rates
x_axis = sorted(error_rates.keys())
y_axis = [error_rates[key] for key in x_axis]
plt.figure()
plt.title("Error Rate by Number of Clusters")
plt.scatter(x_axis, y_axis)
plt.xlabel("Number of Clusters")
plt.ylabel("Error Rate")
plt.show()

/var/folders/7p/w5myjh6j1wl3m52tqz_3fmww0000gn/T/ipykernel_5720/4144908857.py:57: RuntimeWarning: invalid value encountered in double_scalars
  difference_change = abs((max_difference - previous_max_difference) / mean_difference) * 100


## Originial code 

In [ ]:
# writing Lloyd's Algorithm for K-Means clustering.
# (This exists in various libraries, but it's good practice to write by hand.)
def init_centroids(labelled_data, k):
    """
    Randomly pick some k centers from the data as starting values for centroids.
    Remove labels.
    """
    return list(map(lambda x: x[1], random.sample(labelled_data, k)))

# Define a function to sum a cluster element-wise
def sum_cluster(labelled_cluster):
    """
    Element-wise sums a list of arrays. Assumes all datapoints in labelled_cluster are labelled.
    """
    sum_ = labelled_cluster[0][1].copy()
    for (label, vector) in labelled_cluster[1:]:
        sum_ += vector
    return sum_

# Define a function to compute the mean of a cluster
def mean_cluster(labelled_cluster):
    """
    Computes the mean (i.e., the centroid at the middle) of a list of vectors (a cluster).
    Takes the sum and then divides by the size of the cluster.
    Assumes all datapoints in labelled_cluster are labelled.
    """
    sum_of_points = sum_cluster(labelled_cluster)
    mean_of_points = sum_of_points * (1.0 / len(labelled_cluster))
    return mean_of_points

In [ ]:
# Define a function to form clusters
def form_clusters(labelled_data, unlabelled_centroids):
    """
    Given some data and centroids for the data, allocate each datapoint
    to its closest centroid. This forms clusters.
    """
    centroids_indices = range(len(unlabelled_centroids))
    clusters = {c: [] for c in centroids_indices}
    
    for (label, Xi) in labelled_data:
        smallest_distance = float("inf")
        for cj_index in centroids_indices:
            cj = unlabelled_centroids[cj_index]
            distance = np.linalg.norm(Xi - cj)
            if distance < smallest_distance:
                closest_centroid_index = cj_index
                smallest_distance = distance
        clusters[closest_centroid_index].append((label, Xi))
    
    return list(clusters.values())

# Define a function to move centroids
def move_centroids(labelled_clusters):
    """
    Returns a list of centroids corresponding to the clusters.
    """
    new_centroids = []
    for cluster in labelled_clusters:
        new_centroids.append(mean_cluster(cluster))
    return new_centroids

# Define a function to repeat until convergence
def repeat_until_convergence(labelled_data, labelled_clusters, unlabelled_centroids):
    """
    Form clusters around centroids, then keep moving the centroids
    until the moves are no longer significant, i.e., we've found
    the best-fitting centroids for the data.
    """
    previous_max_difference = 0
    while True:
        unlabelled_old_centroids = unlabelled_centroids
        unlabelled_centroids = move_centroids(labelled_clusters)
        labelled_clusters = form_clusters(labelled_data, unlabelled_centroids)
        
        differences = list(map(lambda a, b: np.linalg.norm(a - b), unlabelled_old_centroids, unlabelled_centroids))
        max_difference = max(differences)
        mean_difference = np.mean([previous_max_difference, max_difference])
        
        if mean_difference != 0:
            difference_change = abs((max_difference - previous_max_difference) / mean_difference) * 100
        else:
            difference_change = 0
        
        previous_max_difference = max_difference
        
        if np.isnan(difference_change) or difference_change == 0:
            break
    
    return labelled_clusters, unlabelled_centroids


In [ ]:
def cluster(labelled_data, k):
    """
    runs k-means clustering on the data. It is assumed that the data is labelled.
    """
    centroids = init_centroids(labelled_data, k)
    clusters = form_clusters(labelled_data, centroids)
    final_clusters, final_centroids = repeat_until_convergence(labelled_data, clusters, centroids)
    return final_clusters, final_centroids

In [ ]:
def assign_labels_to_centroids(clusters, centroids):
    """
    Assigns a digit label to each cluster.
    Cluster is a list of clusters containing labelled datapoints.
    NOTE: this function depends on clusters and centroids being in the same order.
    """
    labelled_centroids = []
    for i in range(len(clusters)):
        labels = list(map(lambda x: x[0], clusters[i]))  # Convert map to list
        # Pick the most common label
        most_common = max(set(labels), key=labels.count)
        centroid = (most_common, centroids[i])
        labelled_centroids.append(centroid)
    return labelled_centroids

In [ ]:
def classify_digit(digit, labelled_centroids):
    """
    Given an unlabelled digit represented by a vector and a list of
    labelled centroids [(label, vector)], determine the closest centroid
    and thus classify the digit.
    """
    mindistance = float("inf")
    for (label, centroid) in labelled_centroids:
        distance = np.linalg.norm(centroid - digit)
        if distance < mindistance:
            mindistance = distance
            closest_centroid_label = label
    return closest_centroid_label

def get_error_rate(digits, labelled_centroids):
    """
    Classifies a list of labelled digits. Returns the error rate.
    """
    classified_incorrect = 0
    for (label, digit) in digits:
        classified_label = classify_digit(digit, labelled_centroids)
        if classified_label != label:
            classified_incorrect += 1
    error_rate = classified_incorrect / float(len(digits))
    return error_rate

In [ ]:
# Initialize error rates dictionary
error_rates = {x: None for x in list(range(5, 25)) + [100]}

# Calculate error rates for each value of k
for k in range(5, 25):
    trained_clusters, trained_centroids = cluster(training, k)
    labelled_centroids = assign_labels_to_centroids(trained_clusters, trained_centroids)
    error_rate = get_error_rate(validation, labelled_centroids)
    error_rates[k] = error_rate

# Also calculate for k=100
trained_clusters, trained_centroids = cluster(training, 100)
labelled_centroids = assign_labels_to_centroids(trained_clusters, trained_centroids)
error_rate = get_error_rate(validation, labelled_centroids)
error_rates[100] = error_rate

# Show the error rates
x_axis = sorted(error_rates.keys())
y_axis = [error_rates[key] for key in x_axis]
plt.figure()
plt.title("Error Rate by Number of Clusters")
plt.scatter(x_axis, y_axis)
plt.xlabel("Number of Clusters")
plt.ylabel("Error Rate")
plt.show()


In [ ]:
# Correcting the concatenation of range and list
error_rates = {x: None for x in list(range(5, 25)) + [100]}

for k in range(5, 25):
    trained_clusters, trained_centroids = cluster(training, k)
    labelled_centroids = assign_labels_to_centroids(trained_clusters, trained_centroids)
    error_rate = get_error_rate(validation, labelled_centroids)
    error_rates[k] = error_rate

# Also calculate for k=100
trained_clusters, trained_centroids = cluster(training, 100)
labelled_centroids = assign_labels_to_centroids(trained_clusters, trained_centroids)
error_rate = get_error_rate(validation, labelled_centroids)
error_rates[100] = error_rate

# Show the error rates
x_axis = sorted(error_rates.keys())
y_axis = [error_rates[key] for key in x_axis]
plt.figure()
plt.title("Error Rate by Number of Clusters")
plt.scatter(x_axis, y_axis)
plt.xlabel("Number of Clusters")
plt.ylabel("Error Rate")
plt.show()
